# Разбивка на типы дней и недели
Пример расчета показателей телеканалов в разбивке по типам дней и периодам.

## Описание задачи и условий расчета
- Период: 01.06.2022 - 31.08.2022, отдельно рабочие и не рабочие дни с разбивкой по неделям
- Временной интервал: 05:00-29:00
- ЦА: Все 18+
- Место просмотра: Все места (дом+дача)
- Каналы: НТВ, СТС, ТНТ, РЕН ТВ									
- Статистики: Share

## Инициализация

При построении отчета первый шаг в любом ноутбуке - загрузка библиотек, которые помогут обращаться к TVI API и работать с данными.

Выполните следующую ячейку, для этого перейдите в нее и нажмите Ctrl+Enter

In [1]:
%reload_ext autoreload
%autoreload 2

import sys
import os
import re
import json
import datetime
import time
import pandas as pd
from IPython.display import JSON

from mediascope_api.core import net as mscore
from mediascope_api.mediavortex import tasks as cwt
from mediascope_api.mediavortex import catalogs as cwc

# Настраиваем отображение

# Включаем отображение всех колонок
pd.set_option('display.max_columns', None)
# Задаем максимальное количество выводимых строк. Раскомментируйте нужную строку
# 200 строк
# pd.set_option("display.max_rows", 200)
# Отображаем все строки. ВАЖНО! Отображение большого DataFrame требует много ресурсов
# pd.set_option("display.max_rows", None)

# Cоздаем объекты для работы с TVI API
mnet = mscore.MediascopeApiNetwork()
mtask = cwt.MediaVortexTask()
cats = cwc.MediaVortexCats()

Запрошены записи: 0 - 257
Всего найдено записей: 257

Запрошены записи: 0 - 257
Всего найдено записей: 257



## Справочники

In [2]:
# выберем телесети по имени (части имени)
cats.get_tv_net(name=["НТВ", "СТС", "ТНТ 4", "РЕН ТВ"])

# далее в запросах будем использовать id телесетей.

Запрошены записи: 0 - 0
Всего найдено записей: 0



""


In [3]:
# Найдем названия нужных нам демографических переменных
cats.get_tv_demo_attribute(names=['возраст'])

Запрошены записи: 0 - 125
Всего найдено записей: 125



,id,valueId,name,colName,valueName
0,2,33,Возраст,Age,33
1,16,1,Возраст детей: До 1 года,KidsAge2,Да
2,16,0,Возраст детей: До 1 года,KidsAge2,Нет
3,20,0,Возраст детей: 7-11 лет,KidsAge6,Нет
4,20,1,Возраст детей: 7-11 лет,KidsAge6,Да
...,...,...,...,...,...
120,2,10,Возраст,Age,10
121,2,89,Возраст,Age,89
122,2,8,Возраст,Age,8
123,18,0,Возраст детей: 2-3 года,KidsAge4,Нет


In [4]:
# Найдем идентификаторы нужных нам типов дней
cats.get_tv_research_day_type()

Запрошены записи: 0 - 4
Всего найдено записей: 4



,id,name,ename
0,W,рабочий день,working day
1,H,праздничный день,public holiday
2,E,выходной день,weekend
3,A,день траура,day of mourning


## Формирование задания
Зададим условия расчета

In [5]:
# Период указывается в виде списка ('Начало', 'Конец'). 
date_filter = [('2022-06-01', '2022-08-31')]

# Задаем дни недели
weekday_filter = None

# Задаем тип дня
daytype_filter = None

# Задаем временной интервал
time_filter = None

# Задаем ЦА
basedemo_filter = 'age >= 18'

# Доп фильтр ЦА, нужен только в случае расчета отношения между ЦА, например, при расчете Affinity Index
targetdemo_filter = None

# Задаем место просмотра
location_filter = None

# Задаем каналы
company_filter = 'tvNetId IN (4, 11, 329, 60)'

# Указываем список статистик для расчета
statistics = ['Share']

# Указываем срезы
slices = ['tvNetName', 'researchWeek']

# Задаем опции расчета
options = {
    "kitId": 1, #TV Index Russia all 
}

Сформируем группы, по которым будем делать расчет

In [6]:
groups = {
    'Рабочие дни':'researchDayType = W',
    'Нерабочие дни':'researchDayType IN (E, H)',
}

## Расчет задания

In [7]:
# Посчитаем задания в цикле
tasks = []
print("Отправляем задания на расчет")

# Для каждой группы формируем задания и отправляем на расчет
for name, logical in groups.items():
    
    # Формируем задание на расчет ЦА
    project_name = f"{name}" 
    daytype_filter = f"{logical}"
      
    # Формируем из указанных параметров задание для TVI API в формате JSON для каждого из этапов цикла
    task_json = mtask.build_task('timeband', task_name=project_name, date_filter=date_filter, 
                                 weekday_filter=weekday_filter, daytype_filter=daytype_filter, 
                                 company_filter=company_filter, time_filter=time_filter, 
                                 basedemo_filter=basedemo_filter, targetdemo_filter=targetdemo_filter,
                                 location_filter=location_filter, slices=slices, 
                                 statistics=statistics, options=options)

    # Для каждого этапа цикла формируем словарь с параметрами и отправленным заданием на расчет
    tsk = {}
    tsk['project_name'] = project_name    
    tsk['task'] = mtask.send_timeband_task(task_json)
    tasks.append(tsk)
    time.sleep(2)
    print('.', end = '')

print('')
# Ждем выполнения
print('Ждем выполнения')
tsks = mtask.wait_task(tasks)
print('Расчет завершен, получаем результат')

# Получаем результат
results = []
print('Собираем таблицу')
for t in tasks:
    tsk = t['task'] 
    df_result = mtask.result2table(mtask.get_result(tsk), project_name = t['project_name'])        
    results.append(df_result)
    print('.', end = '')
df = pd.concat(results)
df

Отправляем задания на расчет
..
Ждем выполнения
Расчет задач (2) [ = = = = = = = = = = = = = = ] время расчета: 0:00:45.550653
Расчет завершен, получаем результат
Собираем таблицу
..

,prj_name,researchWeek,tvNetName,Share
0,Рабочие дни,2022-05-30,НТВ,11.203405
1,Рабочие дни,2022-05-30,РЕН ТВ,5.139798
2,Рабочие дни,2022-05-30,СТС,4.000671
3,Рабочие дни,2022-05-30,ТНТ 4,0.403295
4,Рабочие дни,2022-06-06,СТС,3.525606
...,...,...,...,...
47,Нерабочие дни,2022-08-15,НТВ,10.244589
48,Нерабочие дни,2022-08-22,СТС,4.695036
49,Нерабочие дни,2022-08-22,ТНТ 4,0.396612
50,Нерабочие дни,2022-08-22,РЕН ТВ,6.325081


Настраиваем внешний вид таблицы

In [8]:
df_res = pd.pivot_table(df, values=['Share'],
                        index=['prj_name','researchWeek'], 
                        columns=['tvNetName'])
df_res

Share                              
tvNetName                         НТВ    РЕН ТВ       СТС     ТНТ 4
prj_name      researchWeek                                         
Нерабочие дни 2022-05-30     9.667308  6.554655  4.341651  0.405510
              2022-06-06     9.094786  7.013098  4.387012  0.408759
              2022-06-13     9.938344  6.153368  4.226632  0.386506
              2022-06-20     9.243911  5.928410  4.391777  0.407904
              2022-06-27     8.908797  5.974538  3.715142  0.358512
              2022-07-04     8.952596  5.586391  4.134832  0.394088
              2022-07-11     8.546602  6.053147  4.558391  0.299670
              2022-07-18     8.732787  6.174660  4.309734  0.394795
              2022-07-25     7.947439  5.559031  4.005203  0.415800
              2022-08-01     8.952772  5.990097  4.348419  0.426442
              2022-08-08     9.405012  6.236122  4.653612  0.439719
              2022-08-15    10.244589  6.390531  4.415560  0.435210
              2022-08-22     8.792655  6.325081  4.695036  0.396612
Рабочие дни   2022-05-30    11.203405  5.139798  4.000671  0.403295
              2022-06-06    10.820965  5.476063  3.525606  0.377946
              2022-06-13     8.985625  5.678914  3.636372  0.474453
              2022-06-20     9.500732  5.598019  3.737575  0.387810
              2022-06-27    10.045044  5.688982  3.620746  0.376299
              2022-07-04    10.471921  5.823605  3.476181  0.415165
              2022-07-11    10.249489  5.275767  3.853503  0.410390
              2022-07-18    10.232937  6.200180  3.695055  0.433553
              2022-07-25     9.427860  5.897569  3.570758  0.437687
              2022-08-01    10.091796  5.911241  3.798012  0.349707
              2022-08-08    10.466806  5.891221  4.029071  0.355895
              2022-08-15    10.975499  5.943931  4.058303  0.385172
              2022-08-22    11.333313  5.734024  3.973348  0.373119
              2022-08-29    10.323481  5.543956  4.858877  0.387020

## Экспорт в Excel
По умолчанию файл сохраняется в директорию `mediascope-jupyter\mediascope-notebooks\excel`

In [9]:
writer = pd.ExcelWriter(mtask.task_builder.get_excel_filename('02_daytypes&weeks'))
df_info = mtask.task_builder.get_report_info()
df_res.to_excel(writer, 'Report', index=True)
df_info.to_excel(writer, 'Info', index=False)
writer.save()